In [1]:
import sys
import re
import operator
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
from pyspark.sql import SparkSession, functions, types
spark = SparkSession.builder.appName('entity resolution').getOrCreate()
spark.sparkContext.setLogLevel('WARN')
sc = spark.sparkContext
assert spark.version >= '2.3' # make sure we have Spark 2.3+

In [17]:
class EntityResolution:
    def __init__(self, dataFile1, dataFile2, stopWordsFile):
        self.f = open(stopWordsFile, "r")
        self.stopWords = set(self.f.read().split("\n"))
        self.stopWordsBC = sc.broadcast(self.stopWords).value
        self.df1 = spark.read.parquet(dataFile1).cache()
        self.df2 = spark.read.parquet(dataFile2).cache()
    

    def preprocessDF(self, df, cols): 
        df = df.withColumn("joinKey",functions.concat_ws(' ', df[cols[0]], df[cols[1]]))
        df = df.withColumn("joinKey", functions.lower(df.joinKey))
        splitter = functions.udf(lambda x: re.split(r'\W+', x), types.ArrayType(types.StringType()))
        df = df.withColumn("joinKey", splitter(df.joinKey))
        stpWrds = self.stopWordsBC
        stopwords_removal = functions.udf(lambda x: [i for i in x if i not in stpWrds], types.ArrayType(types.StringType()))
        df = df.withColumn("joinKey", stopwords_removal(df.joinKey))
        return df
 
    def filtering(self, df1, df2):
        df1 = df1.withColumn("explosion", functions.explode(df1.joinKey))
        df2 = df2.withColumn("explosion", functions.explode(df2.joinKey))
        candDF = (df2.join(df1, df1.explosion == df2.explosion).select(df1.id.alias("id1"), df1.joinKey.alias("joinKey1"),df2.id.alias("id2"), df2.joinKey.alias("joinKey2")).distinct())
        return candDF

    def verification(self, candDF, threshold):
        def similarity(joinKey1,joinKey2):
            common_items = len(set(joinKey1).intersection(set(joinKey2)))
            if common_items == 0:
                return 0
            elif len(set(joinKey1))+len(set(joinKey2)) ==0:
                return 0
            else:
                return common_items/float(len(set(joinKey1))+len(set(joinKey2))-common_items)
            
        similarity = functions.udf(similarity, types.FloatType())
        resultDF = candDF.withColumn("jaccard", similarity(candDF.joinKey1, candDF.joinKey2))
        return resultDF.where(resultDF.jaccard >= threshold)

    def evaluate(self, result, groundTruth):
        reslength = float(len(result))
        comp_items = [x for x in result if x in groundTruth]
        ci_length = float(len(comp_items))
        precision = ci_length/reslength
        gt_length = len(groundTruth)
        recall = ci_length/gt_length
        fmeasure = 2*precision*recall/(precision+recall)
        return (precision, recall, fmeasure)

    def jaccardJoin(self, cols1, cols2, threshold):
        newDF1 = self.preprocessDF(self.df1, cols1)
        newDF2 = self.preprocessDF(self.df2, cols2)
        print ("Before filtering: %d pairs in total" %(self.df1.count()*self.df2.count())) 

        candDF = self.filtering(newDF1, newDF2)
        print ("After Filtering: %d pairs left" %(candDF.count()))

        resultDF = self.verification(candDF, threshold)
        print ("After Verification: %d similar pairs" %(resultDF.count()))

        return resultDF


    def __del__(self):
        self.f.close()


In [20]:
amazon_input = 'Amazon_sample'
google_input = 'Google_sample'
stopwords_input = 'stopwords.txt'
perfect_mapping_input = 'Amazon_Google_perfectMapping_sample'
er = EntityResolution(amazon_input, google_input, stopwords_input)
amazonCols = ["title", "manufacturer"]
googleCols = ["name", "manufacturer"]
resultDF = er.jaccardJoin(amazonCols, googleCols, 0.5)

result = resultDF.rdd.map(lambda row: (row.id1, row.id2)).collect()
groundTruth = spark.read.parquet(perfect_mapping_input) \
                          .map(lambda row: (row.idAmazon, row.idGoogle)).collect()
print("(precision, recall, fmeasure) = ", er.evaluate(result, groundTruth))

Before filtering: 256 pairs in total


Py4JJavaError: An error occurred while calling o1845.javaToPython.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:146)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:370)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.